In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Preprocessing Data**

In [ ]:
import zipfile
with zipfile.ZipFile("../input/dogs-vs-cats/train.zip","r") as z:
    z.extractall(".")

In [ ]:
with zipfile.ZipFile("../input/dogs-vs-cats/test1.zip",'r') as z:
    z.extractall(".")

> **Load library**

In [ ]:
import os
import skimage.io as io
import random
from skimage.transform import resize
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
path = '/kaggle/working/train/'
f_l = os.listdir(path)
len(f_l)

Sampling due to cpu overload

In [ ]:
sample = random.sample(f_l, int(len(f_l)/10))
sample_image = random.choice(sample)
image = io.imread(path + sample_image)
%matplotlib notebook
plt.imshow(image)

> **Image resize & Cats and Dogs file classification**

In [ ]:
target = []
img_l = []
for file in tqdm(sample):
    img = io.imread(path+file)
    img = resize(img, (150, 150, 3))
    img_l.append(img)
    
    if 'cat' in file:
        target.append(1)
    elif 'dog' in file:
        target.append(0)
        
df_image=pd.DataFrame(
    {"filename":sample,
    "target":target}
)
target = pd.Series(target)

In [ ]:
%matplotlib notebook
plt.imshow(img)

In [ ]:
df_image

In [ ]:
%matplotlib notebook
df_image['target'].value_counts().plot.bar()

In [ ]:
target

# **Data Dimension Reduction**

In [ ]:
retrain = []
for i in range(len(img_l)):
    retrain.append(img_l[i].reshape(150*150*3))
pixel = []
for i in range(150*150*3):
    pixel.append('pixel_{}'.format(i))
df = pd.DataFrame(retrain, columns=pixel)

In [ ]:
df

# **Modeling & Scoring**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

k_fold = KFold(n_splits=5, shuffle = True, random_state = 45)

scores = np.zeros(5)
for i, (tr_idx, te_idx) in enumerate(k_fold.split(df)):
    X_train, y_train = df.iloc[tr_idx], target.iloc[tr_idx]
    x_test, y_test = df.iloc[te_idx], target.iloc[te_idx]
    
    clf = RandomForestClassifier()
    clf.fit(X_train, y_train)
    predict = clf.predict(x_test)
    score = accuracy_score(y_test, predict)
    print(score)
    scores[i] = score

print(round(np.mean(score)*100, 2))